# Predict the next winners of the eurovision contest

## The general algorithm will be:
1) remove any categorical aspects that are not easily codeable(such as song lyrics, song name, etc.)
    -Note, this may be useful in a regex parser for testing what catches a judges eye more.
2) separate the 2009 contestants for validation testing
3) run RandomForestRegressor

In [1]:
import os
import math
import re

import numpy as np

import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype

from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

C:\Users\Aaron\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
PATH = "data/eurovision/"

In [3]:
!ls {PATH}

Eurovision Song Contest 2010 dataset.csv
Eurovision Song Contest 2010 dataset.xls
Final data by year.csv
Semi-final data by year.csv


In [4]:
!head "{PATH}Eurovision Song Contest 2010 dataset.xls"

ÐÏà¡±á                >  þÿ	                               þÿÿÿ        d   å   q  ç  ÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿýÿÿÿB  þÿÿÿC              	   
         
                                                             !   "   #   $   %   &   '   (   )   *   +   ,   -   .   /   0   1   2   3   4   5   6   7   8   9   :   ;   <   =   >   ?   @   A   B   C   D   E   F   G   H   I   J   K   L   M   N   O   P   Q   R   S   T   U   V   W   X   Y   Z   [   \   ]   ^   _   `   a   b   c   e   ýÿÿÿf   g   h   i   j   k   l   m   n   o   p   q   r   s   t   u   v   w   x   y   z   

### Switching to better readability:

In [5]:
!head "{PATH}Eurovision Song Contest 2010 dataset.csv"

Year,Country   ,Region,Song language,Artist   ,Song   ,English translation ,Artist gender,Group/Solo,Place,Points,Host Country,Host region,Home/Away Country,Home/Away Region,Approximate Betting Prices,Albania ,Andorra ,Armenia ,Austria,Azerbaijan ,Belarus ,Belgium ,Bosnia and Herzegovina ,Bulgaria ,Croatia ,Cyprus ,Czech Republic ,Denmark ,Estonia ,Finland ,France ,Georgia,Germany ,Greece ,Hungary ,Iceland ,Ireland ,Israel ,Latvia ,Lithuania ,Macedonia ,Malta ,Moldova ,Monaco,Montenegro ,Netherlands ,Norway ,Poland ,Portugal ,Romania ,Russia ,San Marino,Serbia ,Serbia & Montenegro,Slovakia ,Slovenia ,Spain ,Sweden ,Switzerland ,Turkey ,Ukraine ,United Kingdom ,,,,,,,,,,,,
2009,Lithuania,Former Socialist Bloc,"English, Russian ",Sasha Son ,"""Love"" ",— ,Male,Solo,23,23,Russia,Former Socialist Bloc,Away,Home,270,0,0,0,,1,0,0,0,1,0,0,0,1,2,0,0,,0,0,0,0,7,0,7,,0,0,0,,0,0,0,0,0,0,0,,0,,0,0,0,0,0,0,0,4,,,,,,,,,,,,
2009,Israel,Independent,"English, Hebrew, Arabic ",Noa and Mira Awad ,"""Ther

In [6]:
df_raw = pd.read_csv(f'{PATH}Eurovision Song Contest 2010 dataset.csv', low_memory = False, encoding = 'latin_1',
                   parse_dates = ['Year'])

In [7]:
df_raw

,Year,Country,Region,Song language,Artist,Song,English translation,Artist gender,Group/Solo,Place,...,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74
0,2009-01-01,Lithuania,Former Socialist Bloc,"English, Russian",Sasha Son,"""Love""",,Male,Solo,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-01,Israel,Independent,"English, Hebrew, Arabic",Noa and Mira Awad,"""There Must Be Another Way""",,Female,Group,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-01,France,Western Europe,French,Patricia Kaas,"""Et s'il fallait le faire""",And if it had to be done,Female,Solo,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-01,Sweden,Scandinavia,"English, French",Malena Ernman,"""La voix""",The voice,Female,Solo,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-01,Croatia,Former Yugoslavia,Croatian,Igor Cukrov feat. Andrea,"""Lijepa Tena""",Beautiful Tena,Both,Group,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2009-01-01,Portugal,Western Europe,Portuguese,Flor-de-Lis,"""Todas as ruas do amor""",All the streets of love,Both,Group,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2009-01-01,Iceland,Scandinavia,English,Yohanna,"""Is It True?""",,Female,Solo,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2009-01-01,Greece,Western Europe,English,Sakis Rouvas,"""This Is Our Night""",,Male,Solo,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2009-01-01,Armenia,Former Socialist Bloc,"English, Armenian",Inga and Anush,"""Jan Jan""",My dear,Female,Group,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2009-01-01,Russia,Former Socialist Bloc,"Russian, Ukrainian",Anastasiya Prikhodko,"""Mamo""",Mother,Female,Solo,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### We had to use a different encoding from UTF-8 because the data's encoding is in latin_1. 
### Now first change the Year column to only show the year.

In [8]:
df_raw['Year'].transpose()

0     2009-01-01
1     2009-01-01
2     2009-01-01
3     2009-01-01
4     2009-01-01
5     2009-01-01
6     2009-01-01
7     2009-01-01
8     2009-01-01
9     2009-01-01
10    2009-01-01
11    2009-01-01
12    2009-01-01
13    2009-01-01
14    2009-01-01
15    2009-01-01
16    2009-01-01
17    2009-01-01
18    2009-01-01
19    2009-01-01
20    2009-01-01
21    2009-01-01
22    2009-01-01
23    2009-01-01
24    2009-01-01
25    2008-01-01
26    2008-01-01
27    2008-01-01
28    2008-01-01
29    2008-01-01
         ...    
261   1999-01-01
262   1999-01-01
263   1999-01-01
264   1999-01-01
265   1999-01-01
266   1998-01-01
267   1998-01-01
268   1998-01-01
269   1998-01-01
270   1998-01-01
271   1998-01-01
272   1998-01-01
273   1998-01-01
274   1998-01-01
275   1998-01-01
276   1998-01-01
277   1998-01-01
278   1998-01-01
279   1998-01-01
280   1998-01-01
281   1998-01-01
282   1998-01-01
283   1998-01-01
284   1998-01-01
285   1998-01-01
286   1998-01-01
287   1998-01-01
288   1998-01-

In [9]:
fld = df_raw['Year']

In [10]:
new_column = fld.dt.year

In [11]:
df_raw['Year'] = new_column
df_raw.transpose()

,0,1,2,3,4,5,6,7,8,9,...,281,282,283,284,285,286,287,288,289,290
Year,2009,2009,2009,2009,2009,2009,2009,2009,2009,2009,...,1998,1998,1998,1998,1998,1998,1998,1998,1998,1998
Country,Lithuania,Israel,France,Sweden,Croatia,Portugal,Iceland,Greece,Armenia,Russia,...,United Kingdom,Cyprus,Netherlands,Sweden,Belgium,Finland,Norway,Estonia,Turkey,Macedonia
Region,Former Socialist Bloc,Independent,Western Europe,Scandinavia,Former Yugoslavia,Western Europe,Scandinavia,Western Europe,Former Socialist Bloc,Former Socialist Bloc,...,Independent,Independent,Western Europe,Scandinavia,Western Europe,Independent,Scandinavia,Former Socialist Bloc,Independent,Former Yugoslavia
Song language,"English, Russian","English, Hebrew, Arabic",French,"English, French",Croatian,Portuguese,English,English,"English, Armenian","Russian, Ukrainian",...,English,Greek,Dutch,Swedish,French,Finnish,Norwegian,Estonian,Turkish,Macedonian
Artist,Sasha Son,Noa and Mira Awad,Patricia Kaas,Malena Ernman,Igor Cukrov feat. Andrea,Flor-de-Lis,Yohanna,Sakis Rouvas,Inga and Anush,Anastasiya Prikhodko,...,Imaani,Michalis Hadjiyiannis,Edsilia,Jill Johnson,Mélanie Cohl,Edea,Lars Fredriksen,Koit Toome,Tüzmen,Vlado Janevski
Song,"""Love""","""There Must Be Another Way""","""Et s'il fallait le faire""","""La voix""","""Lijepa Tena""","""Todas as ruas do amor""","""Is It True?""","""This Is Our Night""","""Jan Jan""","""Mamo""",...,"""Where Are You?""","""Genesis""","""Hemel en aarde""","""Kärleken är""","""Dis oui""","""Aava""","""Alltid sommer""","""Mere lapsed""","""Unutamazs?n""","""Ne zori, zoro"""
English translation,,,And if it had to be done,The voice,Beautiful Tena,All the streets of love,,,My dear,Mother,...,,Genesis,Heaven and Earth,Love is,Say yes,Open landscape,Always summer,Children of the sea,You can't forget,"Dawn, don't rise"
Artist gender,Male,Female,Female,Female,Both,Both,Female,Male,Female,Female,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Group/Solo,Solo,Group,Solo,Solo,Group,Group,Solo,Solo,Group,Solo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Place,23,16,8,21,18,15,2,7,10,11,...,2,11,4,10,6,15,8,12,14,19


## Now we will replace the placement numbers with absolute values. The eroor described in the contest was absolute error on the amount of points given to each country. Currently there are some NaN, which we will just turn into 0 for the purpose of this system.

In [12]:
df_raw.fillna(0, inplace = True)
df_raw.head(5)

,Year,Country,Region,Song language,Artist,Song,English translation,Artist gender,Group/Solo,Place,...,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74
0,2009,Lithuania,Former Socialist Bloc,"English, Russian",Sasha Son,"""Love""",,Male,Solo,23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2009,Israel,Independent,"English, Hebrew, Arabic",Noa and Mira Awad,"""There Must Be Another Way""",,Female,Group,16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2009,France,Western Europe,French,Patricia Kaas,"""Et s'il fallait le faire""",And if it had to be done,Female,Solo,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2009,Sweden,Scandinavia,"English, French",Malena Ernman,"""La voix""",The voice,Female,Solo,21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2009,Croatia,Former Yugoslavia,Croatian,Igor Cukrov feat. Andrea,"""Lijepa Tena""",Beautiful Tena,Both,Group,18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## For Song Language, song, translation, we drop. Everything else is extremely relevant, and these are too, but we are just making a simple qualifier right now.

In [13]:
df_raw.drop(columns = ['English translation ', 'Song   '], axis = 1, inplace = True)
df_raw.columns

Index(['Year', 'Country   ', 'Region', 'Song language', 'Artist   ',
       'Artist gender', 'Group/Solo', 'Place', 'Points', 'Host Country',
       'Host region', 'Home/Away Country', 'Home/Away Region',
       'Approximate Betting Prices', 'Albania ', 'Andorra ', 'Armenia ',
       'Austria', 'Azerbaijan ', 'Belarus ', 'Belgium ',
       'Bosnia and Herzegovina ', 'Bulgaria ', 'Croatia ', 'Cyprus ',
       'Czech Republic ', 'Denmark ', 'Estonia ', 'Finland ', 'France ',
       'Georgia', 'Germany ', 'Greece ', 'Hungary ', 'Iceland ', 'Ireland ',
       'Israel ', 'Latvia ', 'Lithuania ', 'Macedonia ', 'Malta ', 'Moldova ',
       'Monaco', 'Montenegro ', 'Netherlands ', 'Norway ', 'Poland ',
       'Portugal ', 'Romania ', 'Russia ', 'San Marino', 'Serbia ',
       'Serbia & Montenegro', 'Slovakia ', 'Slovenia ', 'Spain ', 'Sweden ',
       'Switzerland ', 'Turkey ', 'Ukraine ', 'United Kingdom ', 'Unnamed: 63',
       'Unnamed: 64', 'Unnamed: 65', 'Unnamed: 66', 'Unnamed: 67',
    

In [14]:
df_raw.head(3)

,Year,Country,Region,Song language,Artist,Artist gender,Group/Solo,Place,Points,Host Country,...,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74
0,2009,Lithuania,Former Socialist Bloc,"English, Russian",Sasha Son,Male,Solo,23,23,Russia,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2009,Israel,Independent,"English, Hebrew, Arabic",Noa and Mira Awad,Female,Group,16,53,Russia,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2009,France,Western Europe,French,Patricia Kaas,Female,Solo,8,107,Russia,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Testing if ready for regression!

In [15]:
m = RandomForestRegressor(n_jobs = -1)
m.fit(df_raw.drop('Place', axis= 1),df_raw['Place'])

ValueError: could not convert string to float: 'Lithuania'

### Doesn't work, because the countries need their own coding, the regions also, the song language, the artist, artist gender, Group/Solo, and the host country, Home/Away Country, Home/Away Region

### There's a variable in y that is a string stating "?3", "?5" and so on. Have to remove the prefix and turn those strings into ints. You have to set the categories first since we messed up on missing that earlier

In [17]:
for n, x in df_raw['Place'].items():
    if '?' in x:
        df_raw['Place'][n] = int(x[1::])
df_raw['Place']

C:\Users\Aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0      23
1      16
2       8
3      21
4      18
5      15
6       2
7       7
8      10
9      11
10      3
11      9
12     14
13     22
14      6
15     13
16     20
17      4
18     17
19      1
20     12
21     19
22      5
23     25
24     23
25     20
26     25
27     17
28     23
29      4
       ..
261     5
262    15
263     3
264     7
265     6
266     5
267    20
268    24
269    16
270    25
271    21
272    17
273     1
274     7
275     3
276    23
277    18
278     9
279    12
280    22
281     2
282    11
283     4
284    10
285     6
286    15
287     8
288    12
289    14
290    19
Name: Place, Length: 291, dtype: object

# (More) Feature Engineering
## First, let's handle coding the Group/Solo , artist gender, and song language

In [18]:
def train_cats(df):
    for n, c in df.items():#row, object
        if is_string_dtype(c):#checks if string.
            df[n] = c.astype('category').cat.as_ordered()#sets the categorical value for ordering.

In [19]:
train_cats(df_raw)

## At this point, everything has a categorical code attached now, thanks to the train_cats() function.

In [20]:
df_raw['Group/Solo'].cat.categories

Index([0, 'Group', 'Solo'], dtype='object')

In [21]:
df_raw['Group/Solo'].cat.set_categories([0, 'Solo', 'Group'], ordered = True, inplace = True)

In [22]:
df_raw['Group/Solo'].cat.codes.head(4)

0    1
1    2
2    1
3    1
dtype: int8

In [23]:
df_raw['Artist gender'].cat.categories

Index([0, 'Both', 'Female', 'Male'], dtype='object')

## Now let's try again to see if it will go through.

In [24]:
m.fit(df_raw.drop('Place', axis= 1),df_raw['Place'])

ValueError: could not convert string to float: 'Lithuania'

## Seems that all the variables still need to be switched out with their codes counterpart. We will also put the place column in a separate value.

In [25]:
def numericalize(df, col, name, max_n_cat):
    if not is_numeric_dtype(col) and (max_n_cat is None or col.nunique()>max_n_cat):
        df[name] = col.cat.codes+1# if it isn't numeric, we replace with its codes.

In [26]:
def proc_df(df, y_fld, skip_flds =None, do_scale = False,
           preproc_fn = None, max_n_cat = None, subset = None):
    if not skip_flds:#copies df, grabs y values, drops the original, and then does fix_missing
        skip_flds = []
    if subset:
        df= get_sample(df, subset)
    df = df.copy()
    if preproc_fn:
        preproc_fn(df)
    y = df[y_fld].values
    df.drop(skip_flds+[y_fld], axis=1, inplace =True)
    
    for n, c in df.items():
        fix_missing(df, c, n)
    if do_scale:
        mapper = scale_vars(df)
    for n, c in df.items():
        numericalize(df, c, n, max_n_cat)
    res = [pd.get_dummies(df, dummy_na = True), y]#discuss later
    if not do_scale: return res
    return res + [mapper]

In [27]:
def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum():
            df[name+'_na'] = pd.isnull(col)
        df[name] = col.fillna(col.median())# replaced with median, and replaces with new column telling what's missing

In [28]:
df, y =proc_df(df_raw, 'Place')

In [29]:
y

[23, 16, 8, 21, 18, ..., 15, 8, 12, 14, 19]
Length: 291
Categories (31, object): [1 < 3 < 5 < 6 ... 6 < 7 < 8 < 9]

In [30]:
df.head()

,Year,Country,Region,Song language,Artist,Artist gender,Group/Solo,Points,Host Country,Host region,...,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74
0,2009,24,1,23,221,4,2,23,7,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2009,22,3,16,187,3,3,53,7,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2009,15,5,31,196,3,2,107,7,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2009,39,4,14,146,3,2,33,7,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2009,9,2,6,93,2,3,45,7,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
m = RandomForestRegressor(n_jobs = -1)
m.fit(df, y)
m.score(df, y)

0.9919487982878503

# Final testing/validation

In [39]:
def split_vals(a, n):
    return a[:n].copy(), a[n: ].copy()

n_valid = 20 #same as Kaggle's test size
n_trn = len(df)- n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
Y_train, Y_valid = split_vals(y, n_trn)

X_train.shape, Y_train.shape, X_valid.shape

((271, 72), (271,), (20, 72))

In [50]:
def print_score(m):
    res = [m.predict(X_train), m.predict(X_valid), m.score(X_train, Y_train), m.score(X_valid, Y_valid)]
    if hasattr(m, 'oob_score_'):
        res.append(m.oob_score_)
    print(res)

In [51]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, Y_train)
print_score(m)

Wall time: 109 ms
[array([22.8, 16.6,  8.8, 20.2, 17.1, 15.2,  2.1,  8. , 10.2, 10.9,  3.4,
        9.3, 14.3, 21.3,  7. , 12.8, 19.3,  4. , 17.1,  1. , 12. , 18.7,
        4.9, 24. , 22.7, 18.5, 24.2, 17. , 23.1,  3.8, 10. ,  9.1, 21.3,
       19.1, 23.5, 14.4,  6.5, 13. , 12. , 17.6, 15.2, 11.6,  2. , 20. ,
        9.3,  3.1, 15.8,  6. ,  1.4,  4.6,  9.9, 18.8,  5.8, 24. , 15.7,
       13.8, 14.9,  8.2, 19.7,  6.4, 11.7, 17.8, 22. , 15.6,  2.8, 18.2,
        1.3,  2.1, 22.8, 12.9,  5.4,  4. ,  7.5,  9.8, 16.8, 20.5, 23.3,
       17.1, 15.1, 20.9, 23.9, 15.1, 19.4,  2. , 13.9,  3.4,  3.1,  5. ,
       18.7,  8.6,  1.1,  6. , 22.4, 12. , 10. ,  4.5, 10.6,  7.7,  9.8,
       20.4,  3. ,  3.4,  8.7, 11.5,  5.8, 15.9, 17.8, 20.7,  4.2,  7.1,
        9.8, 18.7, 16.7, 18.7, 24.1, 10.2,  1.1, 14.7, 13.7,  8.1,  5.2,
       22.6, 10.6, 20.6, 24.1, 13.8,  2.7, 12.4, 20. ,  8.5,  8.6,  1.3,
       13.3,  9.6, 22.3, 11.7, 14. ,  2.3, 19. , 22.6, 16.9, 15.9,  5.1,
        4. , 17.9,  4.7,  7.4,  

# 96% success!